# CPUs plot with Altair/Vega

PH, Feb 2025

In [28]:
import pandas as pd
import altair as alt

load CSV export of Baserow table (CSV exported in [Retrieve_baserow.ipynb](Retrieve_baserow.ipynb))

In [29]:
cpus = pd.read_csv('CPUs.csv', parse_dates=['Launch date'])
cpus

,Name,Launch date,Cores,Age,GB6 Single,GB6 Multi,Designer,Win11
0,Intel Core i5-6200U,2015-07-01,2,9.6,916.0,1832.0,Intel,No
1,Intel Core i5-6300U,2015-07-01,2,9.6,965.0,1910.0,Intel,No
2,Intel Core i5-7300U,2017-01-01,2,8.1,1088.0,2030.0,Intel,No
3,Intel Core i5-8250U,2017-07-01,4,7.6,1145.0,3126.0,Intel,Yes
4,Intel Core i5-8350U,2017-07-01,4,7.6,1191.0,3237.0,Intel,Yes
5,Intel Core i5-8365U,2019-04-01,4,5.9,1270.0,3167.0,Intel,Yes
6,Intel Core i5-9400H,2019-04-01,4,5.9,1429.0,4347.0,Intel,Yes
7,Intel Core i5-10210U,2019-07-01,4,5.6,1229.0,3088.0,Intel,Yes
8,Intel Core i5-10310U,2020-04-01,4,4.9,1297.0,3450.0,Intel,Yes
9,Intel Core i5-1135G7,2020-07-01,4,4.6,1702.0,4694.0,Intel,Yes


## Plot

In [33]:
alt.Chart(cpus).mark_point(size=100, filled=True).encode(
    y='Launch date',
    x='GB6 Single',
    tooltip='Name',
    color='Win11',
    shape='Designer'
)

alt.Chart(...)

In [34]:
alt.Chart(cpus).mark_point(size=100, filled=True).encode(
    y='Launch date',
    x='GB6 Multi',
    tooltip='Name',
    color='Win11',
    shape='Designer'
)

alt.Chart(...)